In [1]:
from sklearn.model_selection import cross_val_score

from sklearn import pipeline, preprocessing, compose, feature_selection, linear_model
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn import decomposition
from sklearn.ensemble import RandomForestRegressor
from sklearn import neural_network

from xgboost import XGBRegressor

# Кейс с тестирования магистратуры Тинькофф

Подключаем библиотеки для работы с данными:

In [2]:
import numpy as np
import pandas as pd

np.random.seed(23)

Считаем датасет, переименуем колонки в нормальные имена и заменим знаки вопроса на NaN:

In [3]:
df = pd.read_csv('imports-85.data', header=None)
df.rename(columns={
    0: 'symboling',
    1: 'normalized-losses',
    2: 'make',
    3: 'fuel-type',
    4: 'aspiration',
    5: 'num-of-doors',
    6: 'body-style',
    7: 'drive-wheels',
    8: 'engine-location',
    9: 'wheel-base',
    10: 'length',
    11: 'width',
    12: 'height',
    13: 'curb-weight',
    14: 'engine-type',
    15: 'num-of-cylinders',
    16: 'engine-size',
    17: 'fuel-system',
    18: 'bore',
    19: 'stroke',
    20: 'compression-ratio',
    21: 'horsepower',
    22: 'peak-rpm',
    23: 'city-mpg',
    24: 'highway-mpg',
    25: 'price',
}, inplace=True)
df.replace('?', np.nan, inplace=True)

Изучим датасет, посмотрим сколько есть отсутствующих данных в каждой колонке:

In [4]:
df.isna().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

Заметим, что в колонке normalized-losses есть 41 Nan из 205 данных, т.е. почти 20 процентов. Поэтому исключим этот признак из рассмотрения:

In [5]:
df.drop(['normalized-losses'], axis=1, inplace=True)

Изучим датасет, посмотрев уникальные значения по каждому категориальному признаку

In [6]:
df['make'].value_counts()

toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
volvo            11
peugot           11
dodge             9
bmw               8
mercedes-benz     8
audi              7
plymouth          7
saab              6
porsche           5
isuzu             4
alfa-romero       3
jaguar            3
chevrolet         3
renault           2
mercury           1
Name: make, dtype: int64

Уберем данные с малым количеством цилиндров как выбросы:

In [7]:
df = df.loc[~df['num-of-cylinders'].isin(['twelve', 'three', 'two', 'eight'])]
df = df.loc[~df['num-of-cylinders'].isin(['mercury', 'jaguar', 'chevrolet', 'renault'])]

Разделим датасет на признаки и целевые значения:

In [8]:
X = df.drop(['symboling'], axis=1)
y = df['symboling']

Выделим численные и категориальные признаки:

In [9]:
numeric_columns = [
    'wheel-base', 
    'length',
    'width',
    'height',
    'curb-weight',
    'engine-size',
    'bore',
    'stroke',
    'compression-ratio',
    'horsepower',
    'peak-rpm',
    'city-mpg',
    'highway-mpg',
    'price',
]
categorical_columns = [
    'make',
    'fuel-type',
    'aspiration',
    'num-of-doors',
    'body-style',
    'drive-wheels',
    'engine-location',
    'engine-type',
    'num-of-cylinders',
    'fuel-system',
]

In [10]:
xgb = XGBRegressor(n_estimators=1000, learning_rate=0.1)
nn = neural_network.MLPRegressor(hidden_layer_sizes=(100, 20, 10), activation='relu', solver='lbfgs', alpha=1e+1)

In [11]:
numeric_pipeline = pipeline.Pipeline(steps=[
    ('imputing', SimpleImputer(strategy='mean')),
    ('feature_selection', feature_selection.VarianceThreshold(2.0)),
    ('scaling', preprocessing.StandardScaler()),
])

categorical_pipeline = pipeline.Pipeline(steps=[
    ('imputing', SimpleImputer(strategy='most_frequent')),
    ('onehot_encoding', OneHotEncoder(handle_unknown='ignore')),
    ('feature_selection', feature_selection.SelectKBest(feature_selection.mutual_info_regression, k=34)),
])

column_transformer = compose.ColumnTransformer([
    ('numeric', numeric_pipeline, numeric_columns),
    ('categorical', categorical_pipeline, categorical_columns)
])

estimator = pipeline.Pipeline(steps=[
    ('preprocessing', column_transformer),
    ('model_fitting', xgb),
])

In [12]:
scores = -1 * cross_val_score(estimator, X, y, cv=10, scoring='neg_mean_squared_error')
scores.mean()

0.8176783250571728